## Web Scraping 

https://consult.gov.scot/elections-and-constitutional-development-division/draft-referendum-bill/consultation/view_respondent?uuId=633076343

## Imports

In [41]:
# import sys
# print(sys.executable)
#! /opt/homebrew/Cellar/jupyterlab/3.4.8_1/libexec/bin/python3.11 -m pip install pandas

In [42]:
## Imports
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Parsing the URL of an answer (from a web-from)

In [43]:
# URL of the webpage
url = 'https://consult.gov.scot/elections-and-constitutional-development-division/draft-referendum-bill/consultation/view_respondent?uuId=633076343'

# Send a GET request to the webpage
response = requests.get(url)

# Create a BeautifulSoup object to parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

#print(soup)

## Finding the Questions

In [44]:
#question_elements = soup.find_all('div', class_= "cs-question-padding")
# Find the question and answer elements using appropriate HTML tags and class names
question_elements = soup.find_all('h3', class_= "dss-heading-x-small dss-heading-bold")
#print(question_elements)

questions=[]
for row in question_elements:
    questions.append(row.get_text().strip())
for i in range(0, len(questions)):
    q= questions[i].strip().replace("\n                                                ","")                                          
    char_str = '' .join((z for z in q if not z.isdigit()))
    questions[i]=char_str.replace(".","")
    
    
print(questions)

['What is your name?', 'Are you responding as an individual or an organisation?', 'The Scottish Government would like your permission to publish your consultation response Please indicate your publishing preference:', 'What are your views on the proposed arrangements for managing the referendum?', 'What are your views on the proposed technical changes to polling and count arrangements?']


## Finding the Answers

In [45]:
answer_elements = soup.find_all('div', class_= "cs-question-padding")
#print(answer_elements)



### 1. Name

In [46]:
answers=[]
answer_elements = soup.find_all('div', class_= "cs-fake-text-input")
answers.append(answer_elements[0].get_text().strip())
print(answers)

['Fiona Smith']


### 2 & 3. Radio Buttons: Indiviual/Organization;  Publish info Answers 

In [47]:
radio_answer_elements = soup.find_all('span', class_= "sr-only")
#print(radio_answer_elements[3:])

In [48]:
radio_answers=[]
for row in range(3, len(radio_answer_elements)):
    if "Ticked" in radio_answer_elements[row].get_text().strip():
        radio_answers.append(row-2)
#print(radio_answers)
radio_possible_answers={}
radio_possible_answers[1]="Individual"
radio_possible_answers[2]="Organisation"
radio_possible_answers[3]="Publish response with name"
radio_possible_answers[4]="Publish response only (without name)"
radio_possible_answers[5]="Do not publish response"
for i in radio_answers:
    answers.append(radio_possible_answers[i])
print(answers)

['Fiona Smith', 'Individual', 'Publish response with name']


### 4 & 5. Views

In [49]:
answers_elements = soup.find_all('div', class_= "cs-fake-textarea-input cs-fake-textarea-input-height-10")
#print(answers_elements)

answers_tmp=[]
for row in answers_elements:
    answers.append(row.get_text().strip())

print(answers)

['Fiona Smith', 'Individual', 'Publish response with name', 'I have confidence in the polling arrangements and therefore am happy for them to contiinue . However, more scrutiny within the polling stations  woul alleviate the fears of others. Also more checks should be done that people voting are actually resident and not using family addresses.', 'The  improvements seem sensible.']


In [50]:
print(questions)

['What is your name?', 'Are you responding as an individual or an organisation?', 'The Scottish Government would like your permission to publish your consultation response Please indicate your publishing preference:', 'What are your views on the proposed arrangements for managing the referendum?', 'What are your views on the proposed technical changes to polling and count arrangements?']


## Creating a dataframe with the questions and answers

In [51]:
# Create the DataFrame
df = pd.DataFrame([answers], columns=questions)

# Print the DataFrame
df

,What is your name?,Are you responding as an individual or an organisation?,The Scottish Government would like your permission to publish your consultation response Please indicate your publishing preference:,What are your views on the proposed arrangements for managing the referendum?,What are your views on the proposed technical changes to polling and count arrangements?
0,Fiona Smith,Individual,Publish response with name,I have confidence in the polling arrangements ...,The improvements seem sensible.
